In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler
import sys
import matplotlib.dates as mdates
from datetime import datetime
from scipy import signal
from functools import reduce
from numpy import linspace,cos,sin,pi,exp,angle


class plot_N:

    def __init__(self):
        self.root = "//gridfile/protshar$/nare/My_codes/Moyle_plots/2024_08_20_23/" # Aug_Moyle& EWIC transient
        self.root = "//gridfile/protshar$/nare/My_codes/Moyle_plots/2022_12_13_16_57/" #Kellytown trip
        #self.root = "//gridfile/protshar$/nare/My_codes/Moyle_plots/2024_05_14_01/"
        #self.root = "//gridfile/protshar$/nare/My_codes/Moyle_plots/2024_08_21_23_57_37/"
        #self.root = "//gridfile/protshar$/nare/My_codes/Moyle_plots/2024_07_18_10_39_42/"
        #self.event=datetime(2024, 8, 20, 22, 26, 52, 0) # 1st event
        self.event=datetime(2022, 12, 13, 16, 57, 43, 0) # 1st event
        #self.event=datetime(2024, 5, 14, 1, 0, 47, 0) # 1st event
        
        #self.event=datetime(2024, 8, 21, 23, 57, 37, 0) # 1st event
        #self.event=datetime(2024, 7, 18, 10, 39, 42, 0) # 1st event
        self.colours = ["#0072BD",
                        "#D95319",
                        "#006450",
                        "#7E2F8E",
                        "#77AC30",
                        "#EDB120",
                        "#4DBEEE", 
                        "#A2142F",
                        "#191970",
                        "#A0522D"]
        self.columns = ['IR|Mag',
                    'IS|Mag',
                    'IT|Mag']
        self.columns2=['VR|Mag',
                    'VS|Mag',
                    'VT|Mag']
        self.columns=['Timestamp (dd/MM/yyyy HH:mm:ss.FFF)','3P','3Q','Freq','IR|Angle','IR|Mag','IS|Angle',
                      'IS|Mag','IT|Angle','IT|Mag','VR|Angle','VR|Mag','VS|Angle','VS|Mag','VT|Angle','VT|Mag']
    def run(self):
        list_files = os.listdir(self.root)
        k=0
        cl_df = pd.DataFrame()
        for f in list_files:
            #if(k>5):
            #    break
            df = pd.read_csv(self.root+f)
            df=df[self.columns]
            cols = df.columns.tolist()
            cols[0] = "Time"
            df.columns = cols
            name = reduce(lambda x, y: x+"_"+y, f.split("_")[1:]).split(".csv")[0]
            station_name = name.split(".")[0]
            print(station_name)
            feeder = name.split(".")[-1]
            
                
            alps = [1,exp(1j*2*pi/3),exp(1j*4*pi/3)] #alphas

            def abc2ab(abc,alps): #Clarke transformation, abc to alpha-beta
                  return 2/3*(abc['R']*alps[0]+abc['S']*alps[1]+abc['T']*alps[2])
            def ab2qd(ab,ph_angle): #Park transformation, alpha-beta to d-q
                  return ab*exp(-1j*ph_angle)


            try:
                df['Time'] = pd.to_datetime(df['Time'], infer_datetime_format=True, dayfirst=True)

                tb = self.event - pd.to_timedelta(5, unit="s")
                te = self.event + pd.to_timedelta(5, unit="s") #10
                df = df.set_index("Time")


                df=df.iloc[df.index.get_loc(tb, method='nearest'):df.index.get_loc(te, method='nearest')]
                t=linspace(0,(te-tb).total_seconds(),df.index.shape[0])
                #df['theta']=1*t*2*pi # since sample is per cycle, no point in making in instantenous, require sample time in Khz.
                #df['theta']=50*t*2*pi # fo

                I_angle = df.filter(regex="_I\S|Angle").filter(regex="I").apply(lambda x: np.unwrap(np.radians(x)))
                I_ph_ang=I_angle.mean(axis=1)  
                #I_ph_ang=I_angle[I_angle.columns[0]] #R phase angle 
                I_angle=I_angle.apply(lambda x: exp(1j*x)) 
                I_mag = df.filter(regex="_I\S|Mag").filter(regex="I")

                V_angle = df.filter(regex="_V\S|Angle").filter(regex="V").apply(lambda x: np.unwrap(np.radians(x)))
                V_ph_ang=V_angle.mean(axis=1) # delta_f*t
                #V_ph_ang=V_angle[V_angle.columns[0]] # R phase angle ,delta_f*t
                V_angle=V_angle.apply(lambda x: exp(1j*x))
                V_mag = df.filter(regex="_V\S|Mag").filter(regex="V")
                #df['theta']=df['theta'].apply(lambda x: np.unwrap(np.radians(x)))
                #w_t=df['theta'].apply(lambda x: exp(1j*x)).apply(lambda x:np.real(x))

                cc=['R','S','T']
                I=pd.DataFrame()
                V=pd.DataFrame()
                for c in range (3):
                    I[cc[c]]=I_mag[I_mag.columns[c]]*I_angle[I_angle.columns[c]]
                    V[cc[c]]=V_mag[V_mag.columns[c]]*V_angle[V_angle.columns[c]]
                    df['I_'+cc[c]]=I[cc[c]].apply(lambda x: np.real(x)) # to check plot
                    df['V_'+cc[c]]=V[cc[c]].apply(lambda x: np.real(x)) # to check plot

                Iab=abc2ab(I,alps)
                Idq=ab2qd(Iab,I_ph_ang)
                #Idq=ab2qd(Iab,df['theta']+I_ph_ang)
                Vab=abc2ab(V,alps)
                Vdq=ab2qd(Vab,V_ph_ang)
                #Vdq=ab2qd(Vab,df['theta']+V_ph_ang)
                df['Vab']=Vab
                df['Iab']=Iab
                df['Vdq']=Vdq
                df['Idq']=Idq

                #########################################################################

                #df = df.reset_index(drop=True)
                self._apply(col=2, height=2)
                fig, ax = plt.subplots(nrows=4)
                fig.subplots_adjust(hspace=0.9)  # Adjust this value as needed
                for c in df.filter(regex="_V\S|Mag").filter(regex="V").columns.tolist():
                    #ax[0].plot(df2.index*200/k, df2[c].values*1.732, label=c)
                    ax[0].plot(df.index, df[c].values*1.732, label=c)
                ax[0].set_ylabel("Voltage [kV]")
                ax[0].set_xlabel("Time ")
                leg = ax[0].legend(loc=2)
                leg.get_frame().set_edgecolor('black')
                #ax[0].margins(x=df2.index[0])
                #ax[0].axvline(0, ax[0].get_ylim()[0], ax[0].get_ylim()[1], linestyle="--", color="black")
                ax[0].set_title(station_name+'_'+feeder)


                for c in df.filter(regex="_I\S|Mag").filter(regex="I").columns.tolist():
                    #ax[1].plot(df1.index*200/k, df1[c].values/1000, label=c)
                    ax[1].plot(df.index, df[c].values/1000, label=c)
                ax[1].set_ylabel("Current [kA]")
                ax[1].set_xlabel("Time ")
                leg = ax[1].legend(loc=2)
                leg.get_frame().set_edgecolor('black')
                #ax[1].margins(x=df1.index[0])
                #ax[1].axvline(0, ax[1].get_ylim()[1], ax[1].get_ylim()[1], linestyle="--", color="black")

                ax[2].plot(df.index, (np.real(df['Idq'].values)/3000)-(np.real(df['Idq'].values)/3000)[0], label='Id')
                ax[2].plot(df.index, (np.imag(df['Idq'].values)/3000)-(np.imag(df['Idq'].values)/3000)[0], label='Iq',linestyle='dotted')
                ax[2].set_ylabel("change.I_dq [KA]")
                ax[2].set_xlabel("Time ")
                leg = ax[2].legend(loc=2)
                leg.get_frame().set_edgecolor('black')

                #ax[2].plot(df.index, (np.real(df['Idq'].values)/3000)-(np.real(df['Idq'].values)/3000)[0], label='Id')
                #ax[3].plot(df.index, (np.imag(df['Idq'].values)/3000), label='Iq',color='R')
                #ax[3].set_ylabel("cal.I_q [KA]")
                #ax[3].set_xlabel("Time ")
                #leg = ax[3].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')

                ax[3].plot(df.index, (np.real(df['Vdq'].values))-(np.real(df['Vdq'].values))[0], label='Vd')
                ax[3].plot(df.index, (np.imag(df['Vdq'].values))-(np.imag(df['Vdq'].values))[0], label='Vq',linestyle='dotted')
                ax[3].set_ylabel(" change.V_dq [KV] ")
                ax[3].set_xlabel("Time ")
                leg = ax[3].legend(loc=2)
                leg.get_frame().set_edgecolor('black')
                print('############################')
                
                #ax[4].plot(df.index, (np.real(df['Vdq'].values)/3), label='Vd')
                #ax[4].plot(df.index, (np.imag(df['Vdq'].values)), label='Vq')
                #ax[4].set_ylabel(" cal.V_q [KV] ")
                #ax[4].set_xlabel("Time ")
                #leg = ax[4].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')
                #print('############################')

                #for c in cc:
                #    ax[4].plot(df.index, (df['I_'+c].values)/1000, label='I(t)_'+c)
                #ax[4].set_ylabel(" I(t)[KA] ")
                #ax[4].set_xlabel("Time ")
                #leg = ax[4].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')

                #for c in cc:
                #    ax[3].plot(df.index, (df['V_'+c].values)/1.414, label='V(t)_'+c)
                #ax[3].set_ylabel(" V(t)[KV] ")
                #ax[3].set_xlabel("Time ")
                #leg = ax[3].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')

                #ax[3].plot(df.index, (np.real(df['Vdq'].values)*1.732), label='Vd')
                #ax[3].plot(df.index, (np.imag(df['Vdq'].values)*1.732), label='Vq')
                #ax[3].set_ylabel(" cal.V_dq [KV] ")
                #ax[3].set_xlabel("Time ")
                #leg = ax[3].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')
                print('############################')

                #for c in cc:
                #ax[4].plot(df.index, np.real(df['Vab'].values), label='V(alpha)_'+c)
                #ax[4].set_ylabel("Voltage (alpha) [kV]")
                #ax[4].set_xlabel("Time ")
                #leg = ax[4].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')
                #ax[0].axvline(0, ax[0].get_ylim()[0], ax[0].get_ylim()[1], linestyle="--", color="black")


                #ax[4].plot(df.index, df['3P'].values, label='P')
                #ax[4].set_ylabel(" P [MW] ")
                #ax[4].set_xlabel("Time ")
                #leg = ax[4].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')

                #ax[4].plot(df.index, np.imag(df['Vdq'].values*df['Idq'].values*3)/1000, label='Q')
                #ax[4].set_ylabel(" Q [MVAR] ")
                #ax[4].set_xlabel("Time ")
                #leg = ax[4].legend(loc=2)
                #leg.get_frame().set_edgecolor('black')


                plt.show()
                newpath = "//gridfile/protshar$/nare/My_codes/Moyle_plots/"+str(self.event.date())+'_'+'plots'
                if not os.path.exists(newpath):
                    os.makedirs(newpath)
                fig.savefig(newpath+"/"+station_name+'_'+feeder+".png")

                print()
                k+=1
            except:
                pass
    def _apply(self,
               col=2,
               axis=1,
               width=2,
               height=2,
               borderBottom=1,
               borderLeft=1,
               borderRight=1):
        """col is column width, col = 1 means single column, col = 2 full page width

        axis = 1 means single axis (typically left) axis = 2 means twin axis (twinx)

        width = 1, height = 1 means keep standard width and height, but somethimes
        these need to be adjusted, so there they are, play around.

        borderBottom = 1, is standard border, but increasing this will add extra
        space e.g. to accomodate a large legend or to fit a 3D plot
        """

        plt.rcParams.update(plt.rcParamsDefault)

        def defaults():
            params = { \
                'backend': 'GTK3Agg',
                'font.family': 'serif',
                'font.serif': 'Times',
                'font.sans-serif': ['Helvetica', 'Avant Garde', 'Computer Modern Sans serif'],
                'axes.linewidth': 0.75,
                'text.usetex': False,
                'figure.dpi': 300,
                'savefig.dpi': 600
            }
            return (params)

        def defaultsCol1():

            params = { \
\
                'figure.figsize': (10, 8),

                'figure.subplot.left': 0.155,
                'figure.subplot.right': 0.95,
                'figure.subplot.bottom': 0.15,
                'figure.subplot.top': .95,

                'axes.labelsize': 9,
                'axes.linewidth': .75,

                'font.size': 9,

                'legend.fontsize': 7,
                'xtick.labelsize': 7,
                'ytick.labelsize': 7,

                'lines.linewidth': 1.25
            }

            return (params)

        def defaultsCol2():

            params = { \
\
                'figure.figsize': (10, 8),

                'figure.subplot.left': 0.125,
                'figure.subplot.right': 0.9,
                'figure.subplot.bottom': 0.1,
                'figure.subplot.top': .95,

                'grid.color': 'black',
                'grid.linestyle': '-',
                'grid.linewidth': 0.5,  # in points
                'grid.alpha': 1.0,

                'axes.axisbelow': True,
                'axes.labelsize': 12,

                'font.size': 12,

                'legend.fontsize': 10,
                'xtick.labelsize': 9,
                'ytick.labelsize': 12,

                'lines.linewidth': 2.0
            }

            return (params)

        try:

            params = defaults()
            # print('col = ', col)
            if col == 1:
                params.update(defaultsCol1())
                print('im in col 1')

                if axis == 2:
                    params['figure.subplot.right'] = 0.9


            elif col == 2:
                params.update(defaultsCol2())
                if axis == 2:
                    params['figure.subplot.right'] = 0.875

            if height > 1:
                x, y = params['figure.figsize']
                params['figure.figsize'] = (x, ((height + 1) / 2) * y)
                #print('###########', x, y)
                #print('###########', x, ((height + 1) / 2) * y)

            if width > 1:
                x, y = params['figure.figsize']
                params['figure.figsize'] = (((height + 1) / 2) * x, y)

            if borderBottom != 1:
                params['figure.subplot.bottom'] *= borderBottom

            if borderLeft != 1:
                params['figure.subplot.left'] *= borderLeft

            if borderRight != 1:
                params['figure.subplot.right'] *= borderRight
            df = pd.DataFrame()
            graph_lines = ['-', '--', ':', '-.', '-', ':', '--', '-.']
            colours = self.colours
            for graph_line in graph_lines:
                for col in colours:
                    df = df.append({'Color': col, 'GraphLine': graph_line}, ignore_index=True)
            mpl.rcParams['axes.prop_cycle'] = (
                    cycler(color=df['Color'].tolist()) + cycler(linestyle=df['GraphLine'].tolist()))
            plt.rcParams.update(params)
            plt.rcParams.update(params)

        except:
            print("FAILED TO APPLY IEEE FORMAT_error", sys.exc_info()[:2])


if __name__ == "__main__":
    plot_N = plot_N()
    plot_N.run()
